In [ ]:
!pip install rasterio

In [ ]:
import numpy as np
from shapely.geometry import Polygon, MultiPolygon
from skimage.draw import polygon
from skimage.morphology import erosion, square, binary_erosion
from skimage.io import imread, imsave
from tqdm import tqdm
import rasterio as rs
from math import ceil
from typing import List, Tuple


class Masker:
    def __init__(
        self,
        out_size: Tuple[int, int] = (1024, 1024),
        erosion_kernel: str = 'cross',
        iterator_verbose: bool = True,
    ):
        r"""Initialize Masker object.

        Parameters
        ----------
        out_size : Tuple[int, int]
            Output size of the mask. Default is (1024, 1024).
        erosion_kernel : str
            Type of erosion kernel ('square' or 'cross'). Default is 'cross'.
        iterator_verbose : bool
            Flag to enable verbose mode for iterators. Default is True.
        """        
        self.sz = out_size
        self.pd_sz = (1044, 1044)
        self.x_off = ceil((self.pd_sz[1] - self.sz[1]) / 2)
        self.y_off = ceil((self.pd_sz[0] - self.sz[0]) / 2)
        assert (
            self.x_off >= 0 and self.y_off >= 0
        ), f'out size {self.sz} should be less than padded size {self.pd_sz}'
        assert (
            erosion_kernel.lower() in ['square', 'cross']
        ), f'erosion kernel type: [ {erosion_kernel} ] is not valid'
        self.ek_type = erosion_kernel.lower()
        print('EK TYPE: ',self.ek_type)
        self.itr_vrbs = iterator_verbose
        self.ldir = 'labels_match_pix'

    
    def get_strc(self) -> np.ndarray:
        r"""Get the erosion kernel structure based on the specified type.

        Returns
        -------
        np.ndarray
            Erosion kernel structure.
        """        
        if self.ek_type == 'square':
            return square(3)
        else:
            return np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]], dtype=np.uint8)
    
    
    def make_mask_with_borders(
        self, polys: List[Polygon], size: Tuple[int, int] = (1024, 1024)
    ) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        r"""Generate mask from polygons.

        Parameters
        ----------
        polys : List of polygons
            List of polygons.
        size : Tuple[int, int], optional
            Size of the mask, by default (1024, 1024).

        Returns
        -------
        Tuple[np.ndarray, np.ndarray, np.ndarray]
            Tuple containing instances, buildings, and borders masks.
        """        
        builds, border = [np.zeros(size, dtype=np.uint8), np.zeros(size, dtype=np.uint8)]
        instances = np.zeros(size, dtype=np.int32)
        strc = self.get_strc()
        itr = enumerate(polys)
        if self.itr_vrbs:
            itr = tqdm(itr)
            itr.set_description('generating mask')
        for i, mulpol in itr:
            for j, pol in enumerate(mulpol):
                arr_pol = np.array(pol, dtype=np.int32)
                hs, ws = polygon(arr_pol[:, 0], arr_pol[:, 1], size)
                instances[hs, ws, ...] = np.int32(i + 1) if (j == 0) else 0
            instance = instances == np.int32(i + 1)
            try:
                k = np.where(instance > 0)
                _t = k[0].min() - 2
                _l = k[1].min() - 2
                _b = k[0].max() + 2
                _r = k[1].max() + 2

                crop_instance = instance[_t:_b, _l:_r]
                # print('Structure: ', strc)
                bld = binary_erosion(crop_instance, footprint=strc)
                brdr = bld ^ crop_instance
                brdr1 = np.zeros_like(instance, dtype=brdr.dtype)
                brdr1[_t:_b, _l:_r] = brdr
                border[brdr1 == True] = np.uint8(255)

            except:
                bld = binary_erosion(instance, footprint=strc)
                brdr = bld ^ instance
                border[brdr == True] = np.uint8(255)

        builds[instances > 0] = np.uint8(255)
        # print('\n\ninstances', instances,'\n\nbuilds', builds,'\n\nborder', border)
        return instances, builds, border



In [ ]:
import unittest
import numpy as np
from shapely.geometry import Polygon
from skimage.draw import polygon
from skimage.morphology import square


class TestMasker(unittest.TestCase):

    def test_make_mask_with_borders(self):
        masker = Masker()
        size = (1024, 1024)
        polys = [[[(0.0, 0.0), (100, 0.0), (100, 100), (0.0, 100), (0.0, 0.0)]]]
        
        result_instances, result_builds, result_border = masker.make_mask_with_borders(polys, size)
        
        expected_instances = np.zeros(size, dtype=np.int32)
        expected_builds = np.zeros(size, dtype=np.uint8)
        expected_border = np.zeros(size, dtype=np.uint8)
        
        expected_instances[0:100, 0:100] = 1
        expected_builds[0:100, 0:100] = 255
        expected_border[99:100, 99:100] = 255
        # print('done')
        np.testing.assert_array_equal(result_instances, expected_instances)
        np.testing.assert_array_equal(result_builds, expected_builds)
        np.testing.assert_array_equal(result_border, expected_border)
        # print('done')



unittest.main(argv=['first-arg-is-ignored'], exit=False)